<img src="involys.png" alt="Drawing" style="width: 500px;"/>

# <div style="text-align:center"> PFE : Module de reporting predictif basé sur l'apprentissage machine automatisé </div>

## <div style="text-align:center"> Module 1 : Data Engineering Pipeline </div>

#### <div style="text-align:center"> Le 16 Avril 2020. </div>

## -- I. Overview : 

From raw dataset to a cleaned dataset with exogenous variables.

### ------ Sub-module 1 : File Reading,  Summary, Web Scraping

#### Taches : 

- Lecture de plusieurs types de fichiers (csv, xls, hadoop, json, sql)
- Résumé de la variable cible (moyenne, max, min, NA ..)
- Détection de la nature du probleme (Classification, Régression, Regroupement)
- Détection des types statistiques des variables. (Catégorique, numérique, date)
- Collecte de données exogènes et fusion avec le jeu de donnees brut.

#### Inputs : 

-	Chemin du fichier ou le fichier brut lui meme.  (String ou File) ( e.g : ‘requete.csv )
-	Le nom de la colonne date (String)  ( e.g : ‘date d’execution’ )
-	Le nom de la variable a predire (String) ( e.g : ‘Montant en DHS’ )
-	La ou les variables ID a rejeter (List of Strings) ( e.g : [‘id’ , numero_d’ordre’] )
-	Summary (Boolean) 
-   Petrol (Boolean)
-   Holidays (Boolean) 
-   Temperature (Boolean)
-   Country (String)

#### Output : 


•	Tuple contenant 7 elements : 

-	Nouveau jeu de donnees  (Dataframe) 
-	 Le nom de la colonne date (String)
-	Le nom de la variable a predire (String)
-	Liste des variables numeriques (List of Strings) (e.g : [‘delai’, ‘ temperature’])
-	Liste des variables categoriques (List of Strings) (e.g : [‘nature de pres’, ‘ type budget’])
-	Liste des variables categoriques a en reduire la cardinalite (List of Strings) (e.g : [‘fournisseur’])
-	Type du probleme (String) (e.g : ‘Regression’)



•	Summary (stats)


#### Librairies : 

- pandas : traitement des dataframes et series
- numpy : traitement des arrays, calcul mathematique
- time : calcul du temps de traitement
- BeautifulSoup : html parser for web scraping
- Requests : web scraping
- datetime : Traitement des dates.
- datefinder : detection des dates dans une chaine de caracteres
- google Translator : API pour la traduction
- json : traitement des fichiers json

#### Websites choosen to collect from exogenous data : 

- Holidays : https://www.timeanddate.com/holidays
- Weather : https://www.historique-meteo.net/afrique/
- Petrol : https://finance.yahoo.com/quote/CL%3DF/history?p=CL%3DF

---

### ----- Sub-module 2 : Preprocessing , Cleaning, Time-series techniques.

#### Taches : 

- Traitement des valeurs manquantes (3 méthodes) et valeurs aberrantes.
- Suppression de l'id, des doublons et des variables constantes.
- Prétraitement de la serie temporelle (date, Y) (Stationnarité, saisonnalité, extraction de caractéristiques)
- Création des jeux de donnees pour l'entrainement et pour le test.

#### Inputs : 

-	Output du sous-module 1



#### Output : 

•	Tuple contenant 5 elements : (data_train, data_test, num_features, cat_features, to_reduce)

-	Jeu de donnees pour l’entrainement (DataFrame)
-	Jeu de donnees pour le test (DataFrame)
-	Nouvelle liste des var numeriques (List of strings)
-	Nouvelle liste des var categoriques (List of strings)
-	Nouvelle liste des variables a en reduire la cardinalite. (List of strings)



#### Librairies : 

- pandas : traitement des dataframes et series
- numpy : traitement des arrays, calcul mathematique
- time : calcul du temps de traitement
- statsmodels : statistiques pour les series temporelles

-----

## -- II. Code : 

In [19]:
import pandas as pd
import numpy as np
import time


from bs4 import BeautifulSoup
import requests
import re
import json
import datefinder
from googletrans import Translator
import datetime

### Raw Data to test with

In [60]:
data1 = pd.read_csv('requeteA.csv', sep=",")

In [61]:
data1.head()

,Unnamed: 0,NUMERO_ORDRE,Exercice d’engagement,Date d’engagement,Rubrique budgétaire,Type d’engagement,Montant engagé,Fournisseur,type de dépense,Nature de prestation,Service bénéficiair,"type budget(I,F)"
0,0,50/07,2007,2007-08-04 00:00:00,Achat de fournitures informatiques,Engagement,264360.0,XOS MAROC,Marché Fournitures,Acquisition du consommable informatique,Direction régionale CHAOUIA TADLA (DRCT),Investissement
1,1,673/2008,2008,2008-06-04 00:00:00,Achat de fournitures de bureau,Engagement,8820.0,CARREFOUR TECHNOLOGIE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement
2,2,28/2007,2007,2006-11-26 00:00:00,Agencements et aménagements des constructions,Engagement,441000.0,STE EPICURIA-O'PURE,Marché Fournitures,Acquisition de fournitures sanitaires,Direction des moyens generaux,Investissement
3,3,14/2008,2008,2008-08-19 00:00:00,Achat de fournitures informatiques,Engagement,767520.0,CARREFOUR TECHNOLOGIE,Marché Fournitures,Acquisition du consommable informatique et bur...,Direction des moyens generaux,Investissement
4,4,12/2008,2008,2008-02-14 00:00:00,Achat de fournitures de bureau,Engagement,3360.0,IMPREMERIE HOCHE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement


In [9]:
data2 = pd.read_csv('res2.csv', sep=';', encoding='latin1')

In [10]:
data2.head()

,Date OS,Nature de prestation,Service bnficiaire,Rubrique budgtaire,Fournisseur,type Frs,type de dpense,Montant dpens TTC,Service bnficiair,"type budget(I,F)",dlai dÕexcution,dlai rel dÕexcution.
0,03/09/2011 0:00,Rception et participation aux manifestations ...,Ex Sige,Rception & participation aux manifestations o...,LP TRAITEUR,local,Bon de commandes,11 900,Ex Sige,Investissement,0,NaN
1,07/11/2011 0:00,Acquisition de mobilier de restauration,Ex Sige,"Autres mobiliers, mat.de bureau et amnagement...",ETABLISSEMENT ZITOUNI,local,March Fournitures,1 288 500,Ex Sige,Investissement,2,NaN
2,06/06/2011 0:00,Acquisition du consommable informatique et bur...,Ex Sige,Achat de fournitures informatiques,CARREFOUR TECHNOLOGIE,local,March Fournitures,665 700,Ex Sige,Investissement,2,NaN
3,04/18/2011 00:00:00,Travaux d'impression,Ex Sige,Primes de publicit\t\t\t\t\t\t\t\t\t,IMPRIMAHD,local,Bon de commandes,3 000,Ex Sige,Investissement,0,NaN
4,05/18/2011 00:00:00,Travaux d'impression,Ex Sige,Primes de publicit\t\t\t\t\t\t\t\t\t,MAGHREB COLOR,local,Bon de commandes,54 000,Ex Sige,Investissement,0,NaN


In [11]:
data3 = pd.read_csv('nyc_taxi.csv', sep=',')

In [12]:
data3.head()

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820


In [13]:
data4 = pd.read_csv('hour.csv', sep=",")

In [14]:
data4.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### --------   Sous- Module 1 : Reading, task detection, web scraping, types detection, target summary

#### Fonction : 

In [20]:
def submod1(path, dateColumn, targetColumn , country, separator=None , NA_Column=None, id_=None, summary=True, ) :

    if (path is None) or (targetColumn is None) :

            raise ValueError("You must specify the path or the target column to load the data")

    else:
    
        start_time = time.time()
        
        ################################################ Reading files 
        
        file_type = path.split(".")[-1]
        
        if (file_type == 'csv') :
            
            if (separator is None):
                    raise ValueError("You must specify the separator for a csv file")
            else : 
                print(">> Reading csv file : " + path.split(".")[0] + "  .... ")
                data_erp = pd.read_csv(path, sep=separator,  encoding='latin1', engine='c')
    
        elif  (file_type == 'json') :
            
            if (sys.platform == "win32" and sys.version_info[0] <=3 and sys.version_info[1] <=5):
                raise ValueError("json format not supported for python under 3.6 on windows. Please upgrade python")
                
        
                print(">> Reading json file : " + path.split(".")[-1] + " ...")
                data_erp = pd.read_json(path)
            
        else : 
            
            print(">> Reading excel file : " + path.split(".")[0] + "  .... ")         
            data_erp = pd.read_excel(path)

        df = data_erp.copy()
                  
        ################################################  Drop NA, unnamed, constant columns , Convet date to datetime, Convert target to num.     
        
        if NA_Column != None : 
            df.drop(NA_Column, axis=1, inplace=True)
        if id_ != None and id_ != 'important' :
            df.drop(id_, axis=1, inplace=True)
        if 'Unnamed: 0' in df.columns :
            df.drop('Unnamed: 0', axis=1, inplace=True)
            
        #for col in df.columns : 
           # if df[col].nunique() == 1 : 
                #df.drop(col, axis=1, inplace=True)

        ########## datetime
        
        df[dateColumn] = pd.to_datetime(df[dateColumn], infer_datetime_format=True)

        ######### numeric target
        
        if type(df.loc[0,targetColumn]) == str :
            df[targetColumn] = pd.to_numeric(df[targetColumn].apply(lambda x: x.replace(' ','')))
        else : 
            df[targetColumn] = pd.to_numeric(df[targetColumn])

        df.sort_values(by=dateColumn, inplace=True)

        ################################################  Task : Regression / Classification / MultiClass -- 
        
        
        if (df[targetColumn].nunique() <= 2) :
            type_task = 'Classification'
        elif (df[targetColumn].nunique() <= 10) : 
            type_task = 'MultiClass'
        else :
            type_task = 'Regression'
        
        
        
        
        ################################################  Web Scraping --
        
        def web_scraping(user_data, dateColumn, country) : 
    
                start_time = time.time()
                df = user_data.copy()


            ####### holidays : 

                print("")
                print("Adding holidays data ......")
                translator = Translator()
                country_en = translator.translate(country, dest='en').text.lower()

                url = f'https://www.timeanddate.com/holidays/{country_en}/'
                page = requests.get(url)
                soup = BeautifulSoup(page.content, 'html.parser')
                holidays = []
                for i in range(4,len(soup.find_all('th'))) :
                    holidays.append(soup.find_all('th')[i].text)

                j=0
                for date in holidays :
                    r = datefinder.find_dates(date)
                    for _ in r :
                        holidays[j] = _

                    j+=1

                holidays_df = pd.DataFrame({'holiday date': np.asanyarray(holidays)})
                holidays_df['holiday_month'] = holidays_df['holiday date'].apply(lambda x: x.month)
                holidays_df['holiday_day'] = holidays_df['holiday date'].apply(lambda x: x.day)

                df['is_holiday'] = np.empty((len(df), 0)).tolist()
                for i in range(0, df.shape[0]) :
                    for j in range(0, holidays_df.shape[0]) : 
                        if (df.loc[i, dateColumn].month == holidays_df.loc[j, 'holiday_month']) & (df.loc[i, dateColumn].day == holidays_df.loc[j, 'holiday_day']) :
                            df.loc[i, 'is_holiday'] = 1
                        else :
                            df.loc[i, 'is_holiday'] = 0


            ####### weather : 

             #   print("")
             #   print("Adding weather data ......")

                #url = 'https://www.wunderground.com/history/daily/ma/nouaceur/GMMN/date/2008-3-24'
                #page = requests.get(url)
                #soup = BeautifulSoup(page.content, 'html.parser')


            #    df['temp_moy'] = df[dateColumn]
            #    scrap_months = list(np.arange(1,384,32))
             #   for i in range(0,df.shape[0]) : 

               #     year_data = str(df.loc[i, dateColumn].year)
               #     month_data = df.loc[i, dateColumn].month


                #    if (int(year_data) >= 2009) :

                  #      url = f'https://www.historique-meteo.net/afrique/{country.lower()}/{year_data}/'
                   #     page = requests.get(url)
                   #     soup = BeautifulSoup(page.content, 'html.parser')
                   #     df.loc[i, 'temp_moy'] = soup.find_all('td')[scrap_months[month_data - 1]].text[:2] 

                 #   else :

                 #       df.loc[i, 'temp_moy'] = np.nan
               #     print("Progress :  ", i, '%')


           ####### petrole : 

                print("")
                print("Adding crude-oil data ......")
                
                df['petrole_USD'] = np.empty((len(df), 0)).tolist()
                for i in range(0, df.shape[0]) : 


                    ##convert to posix date integrable in url 

                    cur_date = df.loc[i, dateColumn]
                    cur_date_unix = int(time.mktime(cur_date.timetuple()))
                    next_date = cur_date + datetime.timedelta(days=1)
                    next_date_unix = int(time.mktime(next_date.timetuple()))

                    ## dynamic web scraping

                    url_petrole = f'https://query2.finance.yahoo.com/v8/finance/chart/CL=F?formatted=true&crumb=RoQtzbt66M5&lang=en-US&region=US&interval=1d&period1={cur_date_unix}&period2={next_date_unix}&events=div%7Csplit&corsDomain=finance.yahoo.com'
                    result_p = requests.get(url_petrole, headers={'Referer': 'https://finance.yahoo.com/quote/CL%3DF/history?p=CL%3DF'})

                    adj_close_dict = result_p.json()['chart']['result'][0]['indicators']['adjclose'][0]

                    if len(adj_close_dict) == 0 :

                        df.loc[i, 'petrole_USD'] = np.nan



                    elif len(adj_close_dict['adjclose']) == 1 : 

                        df.loc[i,'petrole_USD'] = adj_close_dict['adjclose'][0]

                    else : 

                        length = len(adj_close_dict['adjclose'])

                        df.loc[i,'petrole_USD'] = adj_close_dict['adjclose'][length - 1]
                        

                df['petrole_USD'] = df['petrole_USD'].astype(float)
                #df['temp_moy'] = df['temp_moy'].astype(float)

                print("")
                print("CPU time for the scraping part : %s seconds" % (time.time() - start_time))

                

                return df
                
                
        print("")
        print(">> Collecting exogenous data .... ")
        df = web_scraping(df, dateColumn, country)
        corr_matrix = df.corr().abs()
        pper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        corr_cols = [column for column in pper.columns if any(pper[column] > 0.70)]
        print("")
        print(pper)
        print("")
        print(corr_cols)
        
        
        ################################################  Collect types --
        
        
        num_features = []
        
        for col in [x for x in df.columns if x != targetColumn] : 
            if (df[col].dtype == float) :
                
                num_features.append(col)
                
                
        
        likely_cat = {}
        cat_features = []
        features_toreduce = []
        for var in [col for col in df.columns if col != dateColumn and col not in num_features ] :
            if ( df[var].nunique()/df[var].count() >  0.10 and df[var].nunique()/df[var].count() < 0.5) :
                features_toreduce.append(var)
            else :    
                likely_cat[var] = 1.*df[var].nunique()/df[var].count() < 0.02
                if likely_cat[var] : 
                    cat_features.append(var)
                elif (df[col].dtype == float) or (df[col].dtype == int) : 
                    num_features.append(var)
                else : 
                    features_toreduce.append(var)

        cat_features = [x for x in cat_features if x not in num_features]
        
        ######### uniques values per categorical column

        uniques_per_catFeat = {}
        for c in cat_features : 
            uniques_per_catFeat[c] = df[c].nunique()
        
        
        
        
        
        
        
        ################################################  Summary -- 
        
        
        if summary : 
            
            print('')
            print(">>>>>>>>>>>> Summary : <<<<<<<<<<<<")
            print('')
            print('>> Number of rows is : ', df.shape[0])
            print('')
            print('>> Number of features is : ', df.shape[1] - 1 )
            print('')
            print(">> Target Variable : ", targetColumn ,  '\n', '\n',  df[targetColumn].describe()[['count', 'mean', 'std', 'min', 'max']].T.round())
            print('')
            print(">> Task type is : ", type_task)
            print('')
            print(">> Missing values : ", '\n', '\n', df.isnull().sum().sort_values(ascending=False))
            print('')
            print(">> Numerical features : ")
            print('')
            print(" Number : ", len(num_features), "numerical features")
            print('')
            print(num_features)
            print('')
            print(">> Categorical features : ")
            print('')
            print(" Number : ", len(cat_features), "categorical features")
            print('')
            print(cat_features)
            print('')
            print(">> Features to reduce : ")
            print('')
            print(" Number : ", len(features_toreduce), "features to reduce")
            print('')
            print(features_toreduce)
            print('')
            print("CPU time for the first sub-module : %s seconds" % (time.time() - start_time))
            
            
            
        return df, dateColumn, targetColumn, cat_features, num_features, features_toreduce, type_task, data_erp

#### Test the function on above datasets : 

In [21]:
req = submod1(path='requeteA.csv', country='Maroc', separator=",", dateColumn='Date dâengagement', targetColumn='Montant engagÃ©', NA_Column=None, id_='NUMERO_ORDRE')

>> Reading csv file : requeteA  .... 

>> Collecting exogenous data .... 

Adding holidays data ......

Adding crude-oil data ......

CPU time for the scraping part : 3749.8844599723816 seconds

                         Exercice dâengagement  Montant engagÃ©  is_holiday  \
Exercice dâengagement                      NaN         0.009684    0.030130   
Montant engagÃ©                              NaN              NaN    0.002123   
is_holiday                                   NaN              NaN         NaN   
petrole_USD                                  NaN              NaN         NaN   

                         petrole_USD  
Exercice dâengagement     0.657638  
Montant engagÃ©             0.020849  
is_holiday                  0.059530  
petrole_USD                      NaN  

[]

>>>>>>>>>>>> Summary : <<<<<<<<<<<<

>> Number of rows is :  8444

>> Number of features is :  11

>> Target Variable :  Montant engagÃ© 
 
 count        8444.0
mean       162692.0
std        943001.

In [16]:
df_2 = submod1(path='res2.csv', separator=';', dateColumn='Date OS', targetColumn='Montant dpens TTC', NA_Column='dlai rel dÕexcution.', id_=None)

Reading csv file : res2  .... 

>> Number of rows is :  8229

>> Number of features is :  10

>> Target Variable :  Montant dpens TTC 
 
 count        8229.0
mean       202655.0
std       1049623.0
min             0.0
max      40000000.0
Name: Montant dpens TTC, dtype: float64

>> Task type is :  Regression

>> Missing values :  
 
 Nature de prestation    179
dlai dÕexcution        59
type budget(I,F)          0
Service bnficiair       0
Montant dpens TTC        0
type de dpense           0
type Frs                  0
Fournisseur               0
Rubrique budgtaire       0
Service bnficiaire      0
Date OS                   0
dtype: int64

>> Numerical features : 

 Number :  0 numerical features

[]

>> Categorical features : 

 Number :  8 categorical features

['Nature de prestation', 'Service b\x8en\x8eficiaire', 'Rubrique budg\x8etaire', 'type Frs', 'type de d\x8epense', 'Service b\x8en\x8eficiair', 'type budget(I,F)', 'd\x8elai dÕex\x8ecution']

>> Features to reduce :

In [17]:
nyc_gdata = submod1(path='nyc_taxi.csv', separator=',', dateColumn='timestamp', targetColumn='value', NA_Column=None, id_=None)

Reading csv file : nyc_taxi  .... 

>> Number of rows is :  10320

>> Number of features is :  1

>> Target Variable :  value 
 
 count    10320.0
mean     15138.0
std       6939.0
min          8.0
max      39197.0
Name: value, dtype: float64

>> Task type is :  Regression

>> Missing values :  
 
 value        0
timestamp    0
dtype: int64

>> Numerical features : 

 Number :  0 numerical features

[]

>> Categorical features : 

 Number :  0 categorical features

[]

>> Features to reduce : 

 Number :  0 features to reduce

[]

CPU time for the first sub-module : 0.04124307632446289 seconds


In [18]:
bikes = submod1(path='hour.csv', separator=',', dateColumn='dteday', targetColumn='cnt', NA_Column='instant')

Reading csv file : hour  .... 

>> Number of rows is :  17379

>> Number of features is :  15

>> Target Variable :  cnt 
 
 count    17379.0
mean       189.0
std        181.0
min          1.0
max        977.0
Name: cnt, dtype: float64

>> Task type is :  Regression

>> Missing values :  
 
 cnt           0
registered    0
casual        0
windspeed     0
hum           0
atemp         0
temp          0
weathersit    0
workingday    0
weekday       0
holiday       0
hr            0
mnth          0
yr            0
season        0
dteday        0
dtype: int64

>> Numerical features : 

 Number :  5 numerical features

['temp', 'atemp', 'hum', 'windspeed', 'registered']

>> Categorical features : 

 Number :  9 categorical features

['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual']

>> Features to reduce : 

 Number :  0 features to reduce

[]

CPU time for the first sub-module : 0.11248898506164551 seconds


### --------  Sous- Module 2 : Train test split , Add dates features , timeseries techniques (Lag-parts , Missing values )

In [75]:
from statsmodels.tsa.seasonal import seasonal_decompose

#### Fonction :  

In [19]:
def submod2(sub_module1, split_size, fill_na) : 
    
    df=sub_module1[0].copy()
    dateColumn = sub_module1[1]
    targetColumn = sub_module1[2]
    cat_features = sub_module1[3].copy()
    num_features = sub_module1[4].copy()
    features_toreduce = sub_module1[5].copy()
    
    X = df.copy().sort_values(by=dateColumn)
     
    start_time = time.time()
    
    def timeseries_train_test_split(X, test_size):
        """
            Perform train-test split with respect to time series structure
        """

        # get the index after which test set starts
        test_index = int(len(X)*(1-test_size))

        X_train = X.iloc[:test_index]
        X_test = X.iloc[test_index:]

        return X_train, X_test
    
    print("Spliting Dataset into train and test ...  ")
    X_train, X_test = timeseries_train_test_split(X, test_size=0.2)
    
    
    def add_process_date(df, dateColumn, targetColumn, seasonal_include=False) : 
        
        """
            Techniques for time-series, feature engineering from time column
        """
        
        new_cats = []
        new_nums = []
        
        if seasonal_include : 
            series = df.reset_index().set_index(dateColumn)[targetColumn].values
            result = seasonal_decompose(series, model='additive', freq=1)

            df['trend'] = result.trend
            df['seasonal'] = result.seasonal
            df['resid'] = result.resid
            df['observed'] = result.observed
            
            new_nums.append('trend','seasonal', 'resid', 'observed' )
            
            print(result.trend, len(result.trend))
            print(result.seasonal, len(result.seasonal))
            print(result.resid, len(result.resid))
            print(result.observed, len(result.observed))
            

        def add_datepart(d, dC, drop=True):
            fld = d[dC]
            new_nums = []
            new_cats = []
            if not np.issubdtype(fld.dtype, np.datetime64):
                d[dC] = fld = pd.to_datetime(fld, 
                                         infer_datetime_format=True)
            targ_pre = dC[:5]
            for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 
                'Dayofyear', 'Is_month_end', 'Is_month_start', 
                'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 
                'Is_year_start'):
                d[targ_pre+n] = getattr(fld.dt,n.lower())
                if n == 'Year' :
                    new_nums.append(targ_pre+n)
                else :
                    new_cats.append(targ_pre+n)
                
            d[targ_pre+'Year'] = d[targ_pre+'Year'].apply(lambda x: int(x-min(d[targ_pre+'Year'])))       
           
            if drop == True :
                d.drop(dC, axis=1, inplace=True)
            
            return d,new_cats,new_nums 

        data_dates = add_datepart(df.reset_index(), dateColumn, drop=False)    
        df = data_dates[0]
        new_cats.extend(data_dates[1])
        new_nums.extend(data_dates[2])
        
        def lag_parts(data, dcol, y):

            new_cats = []
            new_nums = []
            
            data.reset_index()
            data.sort_values(by=dcol)
            data.set_index(dcol, inplace=True)
            target = data[y].astype(int).to_frame()
            for i in range(1, 25):
                data["lag_{}".format(i)] = target[y].shift(i, fill_value=0)
                new_nums.append("lag_{}".format(i))
            return data, new_nums    
        
        lags = lag_parts(df, dateColumn, targetColumn)
        df = lags[0]
        new_nums.extend(lags[1])
        
        df_datedrop = df.reset_index().drop(dateColumn, axis=1)
    
        return df_datedrop, new_cats, new_nums
    
    print("Processing Date Column ....")
    proc_date = add_process_date(X_train, dateColumn, targetColumn)
    X_train = proc_date[0].drop('index', axis=1)
    
    def missing_values (df,  cat_features, num_features, type_fill):
        
        """
            Clean the dataset with the method input as an argument
        """

        
        numCol_with_missing = [col for col in num_features if df[col].isnull().any()]
        catCol_with_missing = [col for col in cat_features if df[col].isnull().any()]
        ##categorical  
        for col in catCol_with_missing :
            if type_fill == 'rm' :
                 df.dropna(inplace=True)
            elif type_fill == 'constant' :
                df[col].fillna('NAA', inplace=True)
            elif type_fill == 'Mode' : 
                df[col].fillna(df[col].mode()[0], inplace=True)
        ##numerical  
        for col in numCol_with_missing :
            if type_fill == 'rm' :
                df.dropna(inplace=True)
            elif type_fill == 'constant' :
                df[col].fillna('-999', inplace=True)
            elif type_fill == 'Median' : 
                df[col].fillna(df[col].mean(), inplace=True)        
        
    print("Dropping Missing values .... " )
    missing_values( X_train, cat_features=sub_module1[3], num_features=sub_module1[4], type_fill=fill_na)
    
    
    categorical, numerical, to_reduce = sub_module1[3].copy(), sub_module1[4].copy(), sub_module1[5].copy()
    categorical.extend(proc_date[1])
    numerical.extend(proc_date[2])
    
  
    print("")
    print("Number of rows for train : ", X_train.shape[0])
    print("Number of rows for test : ", X_test.shape[0])
    print("")
    print("CPU time for the second sub-module : %s seconds" % (time.time() - start_time))
    
    
    
    
    
    return X_train, X_test, categorical, numerical, to_reduce

#### Test the function on above datasets : 

In [20]:
pou = submod2(req, split_size=0.2, fill_na='constant')

Spliting Dataset into train and test ...  
Processing Date Column ....
Dropping Constant and duplicate features .... 

Number of rows for train :  6755
Number of rows for test :  1689

CPU time for the second sub-module : 6.234247922897339 seconds


In [21]:
pou[0].head()

,Exercice dâengagement,Rubrique budgÃ©taire,Type dâengagement,Montant engagÃ©,Fournisseur,type de dÃ©pense,Nature de prestation,Service bÃ©nÃ©ficiair,"type budget(I,F)",Date Year,...,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24
0,2007,Agencements et amÃ©nagements des constructions,Engagement,441000.0,STE EPICURIA-O'PURE,MarchÃ© Fournitures,Acquisition de fournitures sanitaires,Direction des moyens generaux,Investissement,0,...,0,0,0,0,0,0,0,0,0,0
1,2007,Honoraires\t\t\t\t\t\t\t\t\t,Engagement,5940000.0,BDO ASMOUN & ASSOCIES,MarchÃ© Services,Service,Direction des moyens generaux,Investissement,1,...,0,0,0,0,0,0,0,0,0,0
2,2007,Agencements et amÃ©nagements des constructions,Engagement,7302816.0,SEPARATOR,MarchÃ© Travaux,Travaux,Direction des moyens generaux,Investissement,1,...,0,0,0,0,0,0,0,0,0,0
3,2007,Achat de fournitures de bureau,Engagement,129600.0,IMPRIMERIE EL AHDAT,MarchÃ© Fournitures,Acquisition de fournitures de bureau,Direction de la StratÃ©gie,Investissement,1,...,0,0,0,0,0,0,0,0,0,0
4,2007,Achat de fournitures informatiques,Engagement,367200.0,CARREFOUR TECHNOLOGIE,MarchÃ© Fournitures,Acquisition du consommable informatique,Ex SiÃ©ge,Investissement,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
pou2 = submod2(df_2, split_size=0.2, fill_na='constant')

Spliting Dataset into train and test ...  
Processing Date Column ....
Dropping Constant and duplicate features .... 

Number of rows for train :  6583
Number of rows for test :  1646

CPU time for the second sub-module : 6.122455835342407 seconds


In [23]:
pou3 = submod2(nyc_gdata, split_size=0.2, fill_na='constant')

Spliting Dataset into train and test ...  
Processing Date Column ....
Dropping Constant and duplicate features .... 

Number of rows for train :  8256
Number of rows for test :  2064

CPU time for the second sub-module : 8.930663108825684 seconds


In [24]:
pou4 =  submod2(bikes, split_size=0.2, fill_na='constant')

Spliting Dataset into train and test ...  
Processing Date Column ....
Dropping Constant and duplicate features .... 

Number of rows for train :  13903
Number of rows for test :  3476

CPU time for the second sub-module : 28.27296209335327 seconds


###  --------  WEB SCRAPING MODULE

In [56]:
from bs4 import BeautifulSoup
import requests
import re
import json
import datefinder
from googletrans import Translator
import datetime

In [57]:
def web_scraping(user_data, dateColumn, country , weather=False, holidays=False, petrole=False, News=False) : 
    
    start_time = time.time()
    df = user_data.copy()
    
    
    if holidays : 
        
        ## suppose date column is converted to datetime
        
        print("")
        print("Adding holidays data ......")
        translator = Translator()
        country_en = translator.translate(country, dest='en').text.lower()
        
        url = f'https://www.timeanddate.com/holidays/{country_en}/'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        holidays = []
        for i in range(4,len(soup.find_all('th'))) :
            holidays.append(soup.find_all('th')[i].text)

        j=0
        for date in holidays :
            r = datefinder.find_dates(date)
            for _ in r :
                holidays[j] = _

            j+=1

        holidays_df = pd.DataFrame({'holiday date': np.asanyarray(holidays)})
        holidays_df['holiday_month'] = holidays_df['holiday date'].apply(lambda x: x.month)
        holidays_df['holiday_day'] = holidays_df['holiday date'].apply(lambda x: x.day)

        df['is_holiday'] = df[dateColumn]
        for i in range(0, df.shape[0]) :
            for j in range(0, holidays_df.shape[0]) : 
                if (df.loc[i, dateColumn].month == holidays_df.loc[j, 'holiday_month']) & (df.loc[i, dateColumn].day == holidays_df.loc[j, 'holiday_day']) :
                    df.loc[i, 'is_holiday'] = 1
                else :
                    df.loc[i, 'is_holiday'] = 0
    
    
    if weather : 
        
        print("")
        print("Adding weather data ......")
        
        #url = 'https://www.wunderground.com/history/daily/ma/nouaceur/GMMN/date/2008-3-24'
        #page = requests.get(url)
        #soup = BeautifulSoup(page.content, 'html.parser')
        
        
        df['temp_moy'] = df[dateColumn]
        scrap_months = list(np.arange(1,384,32))
        for i in range(0,df.shape[0]) : 

            year_data = str(df.loc[i, dateColumn].year)
            month_data = df.loc[i, dateColumn].month
            

            if (int(year_data) >= 2009) :

                url = f'https://www.historique-meteo.net/afrique/{country.lower()}/{year_data}/'
                page = requests.get(url)
                soup = BeautifulSoup(page.content, 'html.parser')
                df.loc[i, 'temp_moy'] = soup.find_all('td')[scrap_months[month_data - 1]].text[:2] 
            
            else :
    
                df.loc[i, 'temp_moy'] = np.nan
            print("Progress :  ", i, '%')
            
    
    if petrole : 
        
        print("")
        print("Adding petrol data ......")
        df['petrole_USD'] = df[dateColumn]
        for i in range(0, df.shape[0]) : 
            
            
            ##convert to posix date integrable in url 
            
            cur_date = df.loc[i, dateColumn]
            cur_date_unix = int(time.mktime(cur_date.timetuple()))
            next_date = cur_date + datetime.timedelta(days=1)
            next_date_unix = int(time.mktime(next_date.timetuple()))
    
            ## dynamic web scraping
            
            url_petrole = f'https://query2.finance.yahoo.com/v8/finance/chart/CL=F?formatted=true&crumb=RoQtzbt66M5&lang=en-US&region=US&interval=1d&period1={cur_date_unix}&period2={next_date_unix}&events=div%7Csplit&corsDomain=finance.yahoo.com'
            result_p = requests.get(url_petrole, headers={'Referer': 'https://finance.yahoo.com/quote/CL%3DF/history?p=CL%3DF'})
            
            adj_close_dict = result_p.json()['chart']['result'][0]['indicators']['adjclose'][0]
            
            if len(adj_close_dict) == 0 :
                
                df.loc[i, 'petrole_USD'] = np.nan
                
                
                
            elif len(adj_close_dict['adjclose']) == 1 : 
                
                df.loc[i,'petrole_USD'] = adj_close_dict['adjclose'][0]
            
            else : 
                
                length = len(adj_close_dict['adjclose'])
                
                df.loc[i,'petrole_USD'] = adj_close_dict['adjclose'][length - 1]
                
    
    
    
    
    
    
    
    
    
    
    print("")
    print("CPU time for the third sub-module : %s seconds" % (time.time() - start_time))
    
    
    
    
    
    
    
    return df

In [40]:
data1.head()

,Unnamed: 0,NUMERO_ORDRE,Exercice d’engagement,Date d’engagement,Rubrique budgétaire,Type d’engagement,Montant engagé,Fournisseur,type de dépense,Nature de prestation,Service bénéficiair,"type budget(I,F)"
0,0,50/07,2007,2007-08-04 00:00:00,Achat de fournitures informatiques,Engagement,264360.0,XOS MAROC,Marché Fournitures,Acquisition du consommable informatique,Direction régionale CHAOUIA TADLA (DRCT),Investissement
1,1,673/2008,2008,2008-06-04 00:00:00,Achat de fournitures de bureau,Engagement,8820.0,CARREFOUR TECHNOLOGIE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement
2,2,28/2007,2007,2006-11-26 00:00:00,Agencements et aménagements des constructions,Engagement,441000.0,STE EPICURIA-O'PURE,Marché Fournitures,Acquisition de fournitures sanitaires,Direction des moyens generaux,Investissement
3,3,14/2008,2008,2008-08-19 00:00:00,Achat de fournitures informatiques,Engagement,767520.0,CARREFOUR TECHNOLOGIE,Marché Fournitures,Acquisition du consommable informatique et bur...,Direction des moyens generaux,Investissement
4,4,12/2008,2008,2008-02-14 00:00:00,Achat de fournitures de bureau,Engagement,3360.0,IMPREMERIE HOCHE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement


In [54]:
data1['Date d’engagement'] = pd.to_datetime(data1['Date d’engagement'], infer_datetime_format=True)

In [58]:
requete_sc = web_scraping(data1[:100], dateColumn='Date d’engagement', country='Maroc' , weather=False, holidays=True, petrole=True, News=False)


Adding holidays data ......

Adding petrol data ......

CPU time for the third sub-module : 78.468581199646 seconds


In [59]:
requete_sc.head(100)

,Unnamed: 0,NUMERO_ORDRE,Exercice d’engagement,Date d’engagement,Rubrique budgétaire,Type d’engagement,Montant engagé,Fournisseur,type de dépense,Nature de prestation,Service bénéficiair,"type budget(I,F)",is_holiday,petrole_USD
0,0,50/07,2007,2007-08-04,Achat de fournitures informatiques,Engagement,264360.00,XOS MAROC,Marché Fournitures,Acquisition du consommable informatique,Direction régionale CHAOUIA TADLA (DRCT),Investissement,0,NaT
1,1,673/2008,2008,2008-06-04,Achat de fournitures de bureau,Engagement,8820.00,CARREFOUR TECHNOLOGIE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement,0,122.3
2,2,28/2007,2007,2006-11-26,Agencements et aménagements des constructions,Engagement,441000.00,STE EPICURIA-O'PURE,Marché Fournitures,Acquisition de fournitures sanitaires,Direction des moyens generaux,Investissement,0,NaN
3,3,14/2008,2008,2008-08-19,Achat de fournitures informatiques,Engagement,767520.00,CARREFOUR TECHNOLOGIE,Marché Fournitures,Acquisition du consommable informatique et bur...,Direction des moyens generaux,Investissement,0,114.53
4,4,12/2008,2008,2008-02-14,Achat de fournitures de bureau,Engagement,3360.00,IMPREMERIE HOCHE,Bon de commandes,Acquisition de fournitures de bureau,Direction des moyens generaux,Investissement,0,95.46
5,5,51/2008,2008,2009-12-11,Achat de fournitures de bureau,Engagement,90720.00,IMPRIMERIE EL MAARIF EL JADIDA,Marché Travaux,Travaux d'impression,Ex Siége,Investissement,0,69.87
6,6,27/2008,2008,2009-04-13,Mobilier de bureau,Engagement,1395468.00,SMES,Marché Travaux,Acquisition du Mobilier de Bureau,Direction des moyens generaux,Investissement,0,50.05
7,7,42/2008,2008,2009-12-01,Matériel informatique,Engagement,282326.40,IB MAROC.COM,Marché Travaux,Acquisition d'équipements informatiques,Ex Siége,Investissement,0,78.37
8,8,61/2008,2008,2010-02-12,Achats des prestations de service,Engagement,3812400.00,ERNEST & YOUNG,Marché Services,Service,Ex Siége,Investissement,0,74.13
9,9,36/2008,2008,2009-11-09,Immobilisations incorporelles diverses,Engagement,4982140.00,GROUPEMENT DE SOCIETES ALGOE & I3E CONSULTING,Marché Services,Service,Direction des moyens generaux,Investissement,0,79.43


---

---

In [62]:
corr_matrix = data1.corr().abs()

In [63]:
corr_matrix

,Unnamed: 0,Exercice d’engagement,Montant engagé
Unnamed: 0,1.000000,0.114604,0.020521
Exercice d’engagement,0.114604,1.000000,0.009684
Montant engagé,0.020521,0.009684,1.000000


In [64]:
pper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [71]:
[column for column in pper.columns if any(pper[column] > 0.95)]

[]